In [40]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.utils.data

from sklearn.mixture import GaussianMixture 

import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['date'] = pd.to_datetime(data.date)
    print('Days: ',len(set(data.date)))
    return data

In [42]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table,table.index.values

In [43]:
def getMatrixData():
    file = dataDir + dataFile
    dataRaw = loadData(file)
    dataTs,dates = getTimeSeries(dataRaw)
    matrix = dataTs.values
    print('Matrix Shape: ',matrix.shape)
    return matrix,dates

In [44]:
dataDir = '/home/urwa/Documents/Projects/AnomalyDetection/Pipeline/data/'
dataFile = '20190402_TaipeiEdgesDatewise.csv'
events_data =dataDir+'TaipeiEvents.csv'

In [45]:
#matrix = np.stack((dataOut.values, dataIn.values),-1)
matrix,dates = getMatrixData()
matrix = matrix.astype(float)
matrix.shape

Raw shape:  (7374816, 5)
Days:  638
Matrix Shape:  (638, 11664)


(638, 11664)

In [46]:
# for i in range(matrix.shape[1]):
#         matrix[:, i] = (matrix[:, i] - np.mean(matrix[:, i])) / (np.std(matrix[:, i]))

In [47]:
for i in range(matrix.shape[0]):
        matrix[i,:] = (matrix[i,:] - np.mean(matrix[i,:])) / (np.std(matrix[i,:]))

In [48]:
# stations = list(set(dataRaw.start_id))
# n= len(stations)
# edge_index = [[a//n,a%n] for a in range(n*n)]
# edge_index = torch.tensor(edge_index, dtype=torch.long)

In [49]:
#dates = list(dataOut.index)
DOW = list(pd.to_datetime(dates).dayofweek)
DOW = ((np.array(DOW) == 5) | (np.array(DOW) == 6)).astype(int)
DOW[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [50]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(input_dim, hiddenDim[0])
        self.linear2 = nn.Linear(hiddenDim[0], hiddenDim[1])
        self.linear3 = nn.Linear(hiddenDim[1], hiddenDim[2])
        self.linear4 = nn.Linear(hiddenDim[2], n_classes)

        self.init_weights(self.linear1)
        self.init_weights(self.linear2)
        self.init_weights(self.linear3)
        self.init_weights(self.linear4)
        
    def forward(self, x):
        #x, edge_index = data.x, data.edge_index
        x = self.linear1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear4(x)
        return F.log_softmax(x, dim=1)
        
    def init_weights(self,m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def representation(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        return x
            

In [51]:
matrixtensor = torch.tensor(matrix).float()

In [52]:
input_dim = matrix.shape[1]

In [53]:
hiddenDim = [500,100,20]
n_classes = 2

learning_rate=0.001
batch_size = 50
num_epochs = 100

In [54]:
Y = torch.tensor(np.array(DOW), dtype=torch.long)

In [55]:
data_tensor = torch.utils.data.TensorDataset(matrixtensor, Y) 
dataloader = torch.utils.data.DataLoader(dataset = data_tensor, batch_size = batch_size, shuffle = True)

In [56]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = Net().to(device)
#data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [57]:
for epoch in range(num_epochs):
    for data in dataloader:
        X, y = data
        X = X.cuda()
        y = y.cuda()
#         print(X.shape)
#         print(y.shape)
        # ===================forward=====================
        output = model(X)
        #print(output.shape)
        loss = criterion(output, y)
        #MSE_loss = nn.MSELoss()(output, y)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
        .format(epoch + 1, num_epochs, loss.item()))

epoch [1/100], loss:0.4823
epoch [2/100], loss:0.3545
epoch [3/100], loss:0.2921
epoch [4/100], loss:0.1794
epoch [5/100], loss:0.3125
epoch [6/100], loss:0.2256
epoch [7/100], loss:0.1892
epoch [8/100], loss:0.1631
epoch [9/100], loss:0.0671
epoch [10/100], loss:0.1965
epoch [11/100], loss:0.1564
epoch [12/100], loss:0.1086
epoch [13/100], loss:0.0958
epoch [14/100], loss:0.2645
epoch [15/100], loss:0.1323
epoch [16/100], loss:0.1897
epoch [17/100], loss:0.1355
epoch [18/100], loss:0.1797
epoch [19/100], loss:0.0753
epoch [20/100], loss:0.0721
epoch [21/100], loss:0.1079
epoch [22/100], loss:0.1081
epoch [23/100], loss:0.0539
epoch [24/100], loss:0.2064
epoch [25/100], loss:0.1527
epoch [26/100], loss:0.1344
epoch [27/100], loss:0.1364
epoch [28/100], loss:0.2328
epoch [29/100], loss:0.1156
epoch [30/100], loss:0.0747
epoch [31/100], loss:0.1217
epoch [32/100], loss:0.0895
epoch [33/100], loss:0.1424
epoch [34/100], loss:0.0930
epoch [35/100], loss:0.0348
epoch [36/100], loss:0.0542
e

In [58]:
model.eval()
with torch.no_grad():
    y = np.array(DOW)
    predList = model(matrixtensor.cuda()).cpu().numpy()
    pred = np.argmax(predList,axis=1)    
    print('Accuracy: {:.4f}'.format( np.sum(pred == y) / pred.shape[0] ))

Accuracy: 0.9937


In [59]:
with torch.no_grad():
    representation = model.representation(matrixtensor.cuda()).cpu().numpy()
representation.shape

(638, 20)

In [60]:
# import events data
events_data =dataDir+'TaipeiEvents.csv'
df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)


In [61]:
df_events.head()

,Type,Name,Date
0,National holiday,Republic Day/New Year's Day observed,2017-01-02
1,National holiday,Chinese New Year's Eve,2017-01-27
2,National holiday,Chinese New Year's Day,2017-01-28
3,National holiday,Chinese New Year Holiday 1,2017-01-29
4,National holiday,Chinese New Year Holiday 2,2017-01-30


In [62]:
holidayDates = df_events[df_events.Type == 'National holiday'].Date

In [63]:
holidayDates = [str(d.date()) for d in holidayDates]
dates = pd.to_datetime(dates)
dates = [str(d.date()) for d in dates]

In [64]:
anomalyIndex = [i for i,d in enumerate(dates) if d in holidayDates]
len(anomalyIndex)
indexBool = np.array([i in anomalyIndex for i in list(range(matrix.shape[0]))])

In [65]:
def anomalyDetection(y,pval = 0.2,iterN=5,n_com=1):
    #index of regular (non-outlier points)
    #rind=y[:,0]>-10 
    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=n_com, n_init=100, max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if(sum(rind)==0):
            print('All anomalies in {} iterations'.format(i+1))
            break
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [66]:
def getResults(reducedMatrix,threshHolds,iterN=5,n_com=1):
    results = []
    for th in threshHolds:
        #th = thres/100
        print("Threshhold: ",th)
        outliers = anomalyDetection(reducedMatrix,th,iterN,n_com)

        tpr = sum(outliers & indexBool)/sum(indexBool)
        fpr = sum(outliers & ~indexBool)/sum(~indexBool)
        precision = sum(outliers & indexBool)/sum(outliers)

        F1 = 2 * (precision * tpr) / (precision + tpr)

        res = {'Cat':'Global', 'th':th, 'TPR':tpr, 'FPR':fpr, 'F1':F1, 'Precision':precision}
        results.append(res)

    resDf = pd.DataFrame(results)    
    return resDf

In [67]:
threshHolds = [0.01, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [68]:
Res1 = getResults(representation,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.5
I

In [69]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.228571,0.003284,0.666667,0.137931,0.01
1,Global,0.583333,0.008210,0.736842,0.482759,0.03
2,Global,0.592593,0.014778,0.640000,0.551724,0.04
3,Global,0.600000,0.021346,0.580645,0.620690,0.05
4,Global,0.776119,0.019704,0.684211,0.896552,0.06
5,Global,0.712329,0.029557,0.590909,0.896552,0.07
6,Global,0.650000,0.041051,0.509804,0.896552,0.08
7,Global,0.565217,0.060755,0.412698,0.896552,0.10
8,Global,0.346154,0.164204,0.212598,0.931034,0.20
9,Global,0.263636,0.266010,0.151832,1.000000,0.30


In [70]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(matrix)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [71]:
reducedMatrixPCA = pca.transform(matrix)
print(reducedMatrixPCA.shape)

(638, 20)


In [72]:
Res1 = getResults(reducedMatrixPCA,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.5
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Itera

In [73]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.228571,0.003284,0.666667,0.137931,0.01
1,Global,0.416667,0.014778,0.526316,0.344828,0.03
2,Global,0.481481,0.019704,0.520000,0.448276,0.04
3,Global,0.466667,0.027915,0.451613,0.482759,0.05
4,Global,0.447761,0.037767,0.394737,0.517241,0.06
5,Global,0.465753,0.044335,0.386364,0.586207,0.07
6,Global,0.500000,0.050903,0.392157,0.689655,0.08
7,Global,0.434783,0.070608,0.317460,0.689655,0.10
8,Global,0.358974,0.162562,0.220472,0.965517,0.20
9,Global,0.263636,0.266010,0.151832,1.000000,0.30
